In [1]:
import pandas as pd
import requests
import json
import dml
import prov.model
import datetime
import uuid
import csv 
from io import StringIO
import json
import pymongo
import math

In [2]:
# define relational models
def union(R, S):
    return R + S

def difference(R, S):
    return [t for t in R if t not in S]

def intersect(R, S):
    return [t for t in R if t in S]

def project(R, p):
    return [p(t) for t in R]

def select(R, s):
    return [t for t in R if s(t)]
 
def product(R, S):
    return [(t,u) for t in R for u in S]

def aggregate(R, f):
    keys = {r[0] for r in R}
    return [(key, f([v for (k,v) in R if k == key])) for key in keys]


In [3]:
contributor = 'mmao95_Dongyihe_weijiang_zhukk'
reads = []
writes = [contributor + '.street_book']

In [4]:
startTime = datetime.datetime.now()

# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

True

In [8]:
url = 'http://datamechanics.io/data/street-book.csv'
doc = requests.get(url).text
data_file = StringIO(doc)
reader = csv.reader(data_file)

Street_Book_list = []
for row in reader:
    Street_Book_list += [row]

del Street_Book_list[0]

StreetBook = [(full_name, street_name, name_length, gender, gender2, zipcodes, rank) for [num, full_name, street_name, name_length, gender, gender2, zipcodes, rank] in Street_Book_list]

columnName = ['Full Name', 'Street Name', 'Name Length', 'Gender', 'Gender2', 'Zipcodes', 'Rank']
df = pd.DataFrame(columns = columnName, data = StreetBook)
data = json.loads(df.to_json(orient="records"))
print(data)

[{'Full Name': 'A Street,', 'Street Name': 'A ', 'Name Length': '2', 'Gender': 'unknown', 'Gender2': 'F', 'Zipcodes': '02136', 'Rank': '1'}, {'Full Name': 'A Street,', 'Street Name': 'A ', 'Name Length': '2', 'Gender': 'unknown', 'Gender2': 'F', 'Zipcodes': '02210-02127', 'Rank': '2'}, {'Full Name': 'Abbot Street,', 'Street Name': 'Abbot ', 'Name Length': '6', 'Gender': 'male', 'Gender2': 'M', 'Zipcodes': '02124', 'Rank': '2'}, {'Full Name': 'Abbotsford Street,', 'Street Name': 'Abbotsford ', 'Name Length': '11', 'Gender': 'unknown', 'Gender2': 'M', 'Zipcodes': '02121', 'Rank': '1'}, {'Full Name': 'Abby Road,', 'Street Name': 'Abby ', 'Name Length': '5', 'Gender': 'female', 'Gender2': 'M', 'Zipcodes': '02135', 'Rank': '1'}, {'Full Name': 'Aberdeen Street,', 'Street Name': 'Aberdeen ', 'Name Length': '9', 'Gender': 'female', 'Gender2': 'M', 'Zipcodes': '02215', 'Rank': '1'}, {'Full Name': 'Acacia Road,', 'Street Name': 'Acacia ', 'Name Length': '7', 'Gender': 'female', 'Gender2': 'F', '

In [9]:
repo.dropCollection('street_book')
repo.createCollection('street_book')
repo[writes[0]].insert_many(data)

In [10]:
repo[writes[0]].metadata({'complete': True})
print(repo[writes[0]].metadata())
[record for record in repo[writes[0]].find()]

{'_id': ObjectId('5c8153da1877df09197278d7'), 'complete': True}


[{'_id': ObjectId('5c8153c31877df091972652a'),
  'Full Name': 'A Street,',
  'Street Name': 'A ',
  'Name Length': '2',
  'Gender': 'unknown',
  'Gender2': 'F',
  'Zipcodes': '02136',
  'Rank': '1'},
 {'_id': ObjectId('5c8153c31877df091972652b'),
  'Full Name': 'A Street,',
  'Street Name': 'A ',
  'Name Length': '2',
  'Gender': 'unknown',
  'Gender2': 'F',
  'Zipcodes': '02210-02127',
  'Rank': '2'},
 {'_id': ObjectId('5c8153c31877df091972652c'),
  'Full Name': 'Abbot Street,',
  'Street Name': 'Abbot ',
  'Name Length': '6',
  'Gender': 'male',
  'Gender2': 'M',
  'Zipcodes': '02124',
  'Rank': '2'},
 {'_id': ObjectId('5c8153c31877df091972652d'),
  'Full Name': 'Abbotsford Street,',
  'Street Name': 'Abbotsford ',
  'Name Length': '11',
  'Gender': 'unknown',
  'Gender2': 'M',
  'Zipcodes': '02121',
  'Rank': '1'},
 {'_id': ObjectId('5c8153c31877df091972652e'),
  'Full Name': 'Abby Road,',
  'Street Name': 'Abby ',
  'Name Length': '5',
  'Gender': 'female',
  'Gender2': 'M',
  'Zip

In [11]:
repo.logout()
endTime = datetime.datetime.now()
{"start":startTime, "end":endTime}

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Database.logout() is deprecated
  """Entry point for launching an IPython kernel.


{'start': datetime.datetime(2019, 3, 7, 12, 14, 43, 707085),
 'end': datetime.datetime(2019, 3, 7, 12, 24, 55, 821277)}

In [12]:
doc = prov.model.ProvDocument()
startTime = None
endTime = None
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

doc.add_namespace('alg', 'http://datamechanics.io/algorithm/') # The scripts are in <folder>#<filename> format.
doc.add_namespace('dat', 'http://datamechanics.io/data/') # The data sets are in <user>#<collection> format.
doc.add_namespace('ont', 'http://datamechanics.io/ontology#') # 'Extension', 'DataResource', 'DataSet', 'Retrieval', 'Query', or 'Computation'.
doc.add_namespace('log', 'http://datamechanics.io/log/') # The event log.
doc.add_namespace('bdp', 'https://www.50states.com/bio/mass.htm')

this_script = doc.agent('alg:'+contributor+'#street_book', {prov.model.PROV_TYPE:prov.model.PROV['SoftwareAgent'], 'ont:Extension':'py'})
resource = doc.entity('bdp:wc8w-nujj', {'prov:label':'311, Service Requests', prov.model.PROV_TYPE:'ont:DataResource', 'ont:Extension':'json'})
get_names = doc.activity('log:uuid'+str(uuid.uuid4()), startTime, endTime)
doc.wasAssociatedWith(get_names, this_script)
doc.usage(get_names, resource, startTime, None,
    {prov.model.PROV_TYPE:'ont:Retrieval',
    'ont:Computation':'Data cleaning'
    }
)

fp = doc.entity('dat:'+contributor+'#street_book', {prov.model.PROV_LABEL:'Street Book', prov.model.PROV_TYPE:'ont:DataSet'})
doc.wasAttributedTo(fp, this_script)
doc.wasGeneratedBy(fp, get_names, endTime)
doc.wasDerivedFrom(fp, resource, get_names, get_names, get_names)

repo.logout()
doc

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Database.logout() is deprecated


<ProvDocument>

In [13]:
print(doc.get_provn())
print(json.dumps(json.loads(doc.serialize()), indent=4))

document
  prefix alg <http://datamechanics.io/algorithm/>
  prefix dat <http://datamechanics.io/data/>
  prefix ont <http://datamechanics.io/ontology#>
  prefix log <http://datamechanics.io/log/>
  prefix bdp <https://www.50states.com/bio/mass.htm>
  
  agent(alg:mmao95_Dongyihe_weijiang_zhukk#street_book, [prov:type='prov:SoftwareAgent', ont:Extension="py"])
  entity(bdp:wc8w-nujj, [prov:label="311, Service Requests", prov:type="ont:DataResource", ont:Extension="json"])
  activity(log:uuid0f6e0979-a389-4996-b63c-db5050fcc6af, -, -)
  wasAssociatedWith(log:uuid0f6e0979-a389-4996-b63c-db5050fcc6af, alg:mmao95_Dongyihe_weijiang_zhukk#street_book, -)
  used(log:uuid0f6e0979-a389-4996-b63c-db5050fcc6af, bdp:wc8w-nujj, -, [prov:type="ont:Retrieval", ont:Computation="Data cleaning"])
  entity(dat:mmao95_Dongyihe_weijiang_zhukk#street_book, [prov:label="Street Book", prov:type="ont:DataSet"])
  wasAttributedTo(dat:mmao95_Dongyihe_weijiang_zhukk#street_book, alg:mmao95_Dongyihe_weijiang_zhukk